# BigQuery Feature Extraction Example

This notebook demonstrates how to use the `BigQueryFeatureExtractor` to extract features from Google BigQuery datasets for use with Ember ML models.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from ember_ml.nn import tensor
from ember_ml import ops
from ember_ml.nn.features.bigquery_feature_extractor import BigQueryFeatureExtractor

## 1. Initialize the Feature Extractor

First, we need to initialize the BigQuery feature extractor with our project, dataset, and table information. If you have a service account key file, you can provide the path to it in the `credentials_path` parameter.

In [ ]:
# Initialize the feature extractor
extractor = BigQueryFeatureExtractor(
    project_id='your-project-id',
    dataset_id='your-dataset-id',
    table_id='your-table-id',
    credentials_path='path/to/your/credentials.json'  # Optional
)

## 2. Auto-detect Column Types

We can automatically detect and categorize columns as numeric, categorical, or datetime based on their BigQuery data types.

In [ ]:
# Auto-detect column types
extractor.auto_detect_column_types()

# Print the detected column types
print("Numeric columns:", extractor.numeric_columns)
print("Categorical columns:", extractor.categorical_columns)
print("Datetime columns:", extractor.datetime_columns)

## 3. Fetch Data

Now we can fetch data from the BigQuery table. We can limit the number of rows, apply filters, and sort the data as needed.

In [ ]:
# Fetch data with limit and filters
data = extractor.fetch_data(
    limit=1000,                  # Optional: Limit the number of rows
    where_clause="column1 > 0",  # Optional: Filter the data
    order_by="column2 DESC"      # Optional: Sort the data
)

# Display a sample of the data
data.head()

## 4. Extract Features

Now we can extract features from the data. The feature extractor will process numeric, categorical, and datetime features and combine them into a single tensor.

In [ ]:
# Extract features
features, feature_names = extractor.extract_features(
    data=data,                   # Use the fetched data
    handle_missing=True,         # Handle missing values
    handle_outliers=True,        # Handle outliers
    normalize=True,              # Normalize numeric features
    create_samples=True,         # Create sample tables for visualization
    capture_processing=True      # Capture processing steps for animation
)

# Print feature information
print(f"Features shape: {tensor.shape(features)}")
print(f"Number of features: {len(feature_names)}")
print(f"Feature names: {feature_names[:10]}...")

## 5. Visualize Sample Data

We can visualize the sample data that was captured during feature extraction.

In [ ]:
# Display raw data sample
pd.DataFrame(extractor.sample_tables['raw_data']['data'])

In [ ]:
# Display processed features sample
pd.DataFrame(extractor.sample_tables['processed_features']['data'])

## 6. Use the Features in a Model

Now that we have extracted the features, we can use them in an Ember ML model. Here's a simple example using a neural network.

In [ ]:
from ember_ml.nn.modules import Sequential
from ember_ml.nn.container import Linear
from ember_ml.nn.modules.activations import ReLU, Sigmoid

# Create a simple neural network
model = Sequential([
    Linear(tensor.shape(features)[1], 64),  # Input layer
    ReLU(),
    Linear(64, 32),                      # Hidden layer
    ReLU(),
    Linear(32, 1),                       # Output layer
    Sigmoid()
])

# Forward pass
output = model(features)
print(f"Output shape: {tensor.shape(output)}")

## 7. Execute Custom Queries

You can also execute custom SQL queries directly on the BigQuery table.

In [ ]:
# Execute a custom query
query = """
SELECT
  column1,
  AVG(column2) as avg_column2,
  COUNT(*) as count
FROM
  `your-project-id.your-dataset-id.your-table-id`
GROUP BY
  column1
ORDER BY
  count DESC
LIMIT
  10
"""

# Execute the query
result = extractor.execute_query(query)

# Display the result
result

## 8. Working with Multiple Tables

You can also create multiple feature extractors for different tables and combine their features.

In [ ]:
# Create a second feature extractor
extractor2 = BigQueryFeatureExtractor(
    project_id='your-project-id',
    dataset_id='your-dataset-id',
    table_id='your-second-table-id',
    credentials_path='path/to/your/credentials.json'  # Optional
)

# Auto-detect column types
extractor2.auto_detect_column_types()

# Fetch data
data2 = extractor2.fetch_data(limit=1000)

# Extract features
features2, feature_names2 = extractor2.extract_features(data=data2)

# Combine features
all_features = ops.concatenate([features, features2], axis=1)
all_feature_names = feature_names + feature_names2

print(f"Combined features shape: {tensor.shape(all_features)}")
print(f"Total number of features: {len(all_feature_names)}")

## 9. Advanced Usage: Custom Feature Processing

For more advanced feature processing, you can use the functions from the `bigquery` package directly.

In [ ]:
from ember_ml.nn.features.bigquery import (
    process_numeric_features,
    process_categorical_features,
    normalize_robust,
    hash_encode
)

# Custom numeric feature processing
custom_numeric_tensor, custom_numeric_names = process_numeric_features(
    df=data,
    numeric_columns=['custom_column1', 'custom_column2'],
    handle_missing=True,
    handle_outliers=False,  # Skip outlier handling
    normalize=True
)

# Custom hash encoding for a high-cardinality column
hash_tensor, hash_names = hash_encode(
    df=data,
    column='high_cardinality_column',
    n_components=32  # Use 32 hash components
)

# Combine custom features
custom_features = ops.concatenate([custom_numeric_tensor, hash_tensor], axis=1)
custom_feature_names = custom_numeric_names + hash_names

print(f"Custom features shape: {tensor.shape(custom_features)}")
print(f"Number of custom features: {len(custom_feature_names)}")